# Installing and Importing Libraries

In [ ]:
# Install Required Libraries
!pip install --upgrade pip
!pip install torch torchvision torchaudio
!pip install ultralytics
!pip install opencv-python-headless matplotlib tqdm pandas scikit-learn seaborn pillow

In [ ]:
# Import necessary libraries
import pandas as pd
import os
import shutil
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

# Loading and Exploring Dataset

In [ ]:
# Path to the dataset
dataset_path = "/kaggle/input/gtsrb-german-traffic-sign"
print(os.listdir(dataset_path))


In [ ]:
# Load the training labels from the CSV file
train_labels_path = '/kaggle/input/gtsrb-german-traffic-sign/Train.csv'  
train_labels = pd.read_csv(train_labels_path)

# Display the first few rows of the labels
print(train_labels.head())

# Data Augmentation

# Splitting Dataset into Training and Testing Datasets

In [ ]:
import os
import cv2
import pandas as pd
from tqdm import tqdm

# Paths
dataset_path   = "/kaggle/input/gtsrb-german-traffic-sign"
train_csv_path = os.path.join(dataset_path, "Train.csv")

images_dir = "images"
labels_dir = "labels"
os.makedirs(images_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)

# Load CSV
train_labels = pd.read_csv(train_csv_path)

# Iterate over rows
for idx, row in tqdm(train_labels.iterrows(), total=len(train_labels)):
    img_path = os.path.join(dataset_path, row["Path"])
    image    = cv2.imread(img_path)
    if image is None:
        continue

    # (Optional) apply your augmentation
    # aug = apply_all_effects(image)
    aug = image  

    # Save image with new sequential name
    img_name = f"{idx:06d}.png"
    cv2.imwrite(os.path.join(images_dir, img_name), aug)

    # Extract YOLO-format label
    x1, y1, x2, y2 = row["Roi.X1"], row["Roi.Y1"], row["Roi.X2"], row["Roi.Y2"]
    w_img, h_img   = row["Width"], row["Height"]

    # Normalize
    x_c = ((x1 + x2) / 2) / w_img
    y_c = ((y1 + y2) / 2) / h_img
    bw  = (x2 - x1) / w_img
    bh  = (y2 - y1) / h_img

    label_txt = f"{row['ClassId']} {x_c:.6f} {y_c:.6f} {bw:.6f} {bh:.6f}\n"

    # Save label file
    with open(os.path.join(labels_dir, f"{idx:06d}.txt"), "w") as f:
        f.write(label_txt)

print(f"✅ Done. Saved {len(train_labels)} images and labels.")

In [ ]:
base_out = "/kaggle/working/"

train_img_dir  = os.path.join(base_out, "train/images")
train_lbl_dir  = os.path.join(base_out, "train/labels")
test_img_dir   = os.path.join(base_out, "test/images")
test_lbl_dir   = os.path.join(base_out, "test/labels")
for d in [train_img_dir, train_lbl_dir, test_img_dir, test_lbl_dir]:
    os.makedirs(d, exist_ok=True)

In [ ]:
import random
# =========================
# TRAIN/TEST SPLIT
# =========================
temp_images_dir = "/kaggle/working/images"
temp_labels_dir = "/kaggle/working/labels"

test_ratio = 0.2
all_imgs   = [f for f in os.listdir(temp_images_dir) if f.endswith(".png")]
random.shuffle(all_imgs)
split_idx  = int(len(all_imgs) * (1 - test_ratio))
train_set, test_set = all_imgs[:split_idx], all_imgs[split_idx:]

def copy_split(files, dest_img_dir, dest_lbl_dir):
    for fname in tqdm(files, desc=f"Copying to {dest_img_dir}"):
        shutil.copy(os.path.join(temp_images_dir, fname), os.path.join(dest_img_dir, fname))
        label_file = fname.replace(".png", ".txt")
        shutil.copy(os.path.join(temp_labels_dir, label_file), os.path.join(dest_lbl_dir, label_file))

copy_split(train_set, train_img_dir, train_lbl_dir)
copy_split(test_set,  test_img_dir,  test_lbl_dir)

print("✅ Combined-effects dataset ready at:", base_out)

In [ ]:
len(os.listdir("/kaggle/working/test/images"))+len(os.listdir("/kaggle/working/train/images")) 

## Creating YAML File for training

In [ ]:
import yaml

# Base directory where the train/test folders were created
base_out = "/kaggle/working/"

# Paths to train/test image folders
train_images_path = os.path.join(base_out, "train/images")
val_images_path   = os.path.join(base_out, "test/images")

# YAML file to save
yaml_file_path = "/kaggle/working/combined_effects.yaml"

# Number of classes (GTSRB has 43)
num_classes = 43

# If you have the real class names, replace the list below
class_names = [f"Class_{i}" for i in range(num_classes)]

# Dataset info
dataset_info = {
    "train": train_images_path,
    "val":   val_images_path,
    "nc":    num_classes,
    "names": class_names
}

# Write YAML
with open(yaml_file_path, "w") as f:
    yaml.dump(dataset_info, f, default_flow_style=False)

print(f"✅ YOLO dataset YAML saved to {yaml_file_path}")

In [ ]:

base = "/kaggle/working"

# Create train/test subdirectories in the base directory
for sub in ["train/images", "train/labels", "test/images", "test/labels"]:
    os.makedirs(os.path.join(base, sub), exist_ok=True)

imgs = [f for f in os.listdir(os.path.join(base, "images")) if f.endswith(".png")]
random.shuffle(imgs)
cut = int(0.8 * len(imgs))
train_imgs, test_imgs = imgs[:cut], imgs[cut:]

def move(files, dest):
    for f in tqdm(files):
        shutil.copy(os.path.join(base, "images", f), os.path.join(base, f"{dest}/images", f))
        label = f.replace(".png", ".txt")
        shutil.copy(os.path.join(base, "labels", label), os.path.join(base, f"{dest}/labels", label))

move(train_imgs, "train")
move(test_imgs, "test")

# Training YOLOV8 Model

In [ ]:
from ultralytics import YOLO
import os

DATA_PATH = "/kaggle/working/combined_effects.yaml"
MODEL_PATH = "/kaggle/working/yolov8n.pt"
PROJECT_PATH = "/kaggle/working/runs"
RUN_NAME = "detect_train_stable"
EPOCH_SEGMENT = 10
TARGET_EPOCHS = 50
BATCH_SIZE = 8
IMG_SIZE = 512

ckpt_path = os.path.join(PROJECT_PATH, RUN_NAME, "weights", "last.pt")
current_epoch = 0

while current_epoch < TARGET_EPOCHS:
    next_epoch = min(current_epoch + EPOCH_SEGMENT, TARGET_EPOCHS)
    print(f"\n🔁 Training from epoch {current_epoch + 1} up to {next_epoch}...\n")

    # ✅ Reload YOLO model fresh each time
    if os.path.exists(ckpt_path):
        print(f"✅ Resuming weights from checkpoint: {ckpt_path}")
        model = YOLO(ckpt_path)
    else:
        model = YOLO(MODEL_PATH)

    # ✅ Notice: no "resume=True" now
    model.train(
        data=DATA_PATH,
        epochs=next_epoch,      # total target, YOLO will continue till this
        imgsz=IMG_SIZE,
        batch=BATCH_SIZE,
        device=0,
        deterministic=True,
        project=PROJECT_PATH,
        name=RUN_NAME,
        exist_ok=True
    )

    current_epoch = next_epoch
    ckpt_path = os.path.join(PROJECT_PATH, RUN_NAME, "weights", "last.pt")

print("\n✅ Stable multi-stage YOLO training finished successfully!")

## Evaluating YOLOV8 model

In [ ]:
from ultralytics import YOLO
model = YOLO("/kaggle/working/runs/detect_train_stable/weights/last.pt")  

# Run the evaluation
results = model.val(data="combined_effects.yaml")
# Print overall metrics
print("Overall Metrics:")
print(f"Precision: {results.box.p.mean():.4f}")  # Overall precision
print(f"Recall: {results.box.r.mean():.4f}")      # Overall recall
print(f"Mean Average Precision (mAP): {results.box.map.mean():.4f}")  # mAP
print(f"Mean Average Precision at IoU=0.50: {results.box.map50.mean():.4f}")  # mAP at IoU=0.50
print(f"Mean Average Precision at IoU=0.75: {results.box.map75.mean():.4f}")  # mAP at IoU=0.75
print(f"F1 Score: {results.box.f1.mean():.4f}")  # F1 score

In [ ]:
import os
import shutil
from ultralytics import YOLO

# ---------- YOLOv8 Export ----------

trained_yolo_weights = '/kaggle/working/runs/detect_train_stable/weights/last.pt'
yolo_model = YOLO(trained_yolo_weights)

# Export to ONNX
yolo_model.export(
    format='onnx',
    opset=12,
    simplify=True,
    dynamic=True
)

# Correct exported file path
onnx_exported = '/kaggle/working/runs/detect_train_stable/weights/last.onnx'
onnx_final = '/kaggle/working/yolov8n.onnx'

if os.path.exists(onnx_exported):
    shutil.copy2(onnx_exported, onnx_final)
    print(f"✅ YOLO ONNX copied to: {onnx_final}")
else:
    print("⚠️ ONNX export file not found — check logs.")